## BIA Tischi demo
BIA Tischi is a Jupyter-magic based assistant for interacting data via code

In [1]:
from bia_tischi import tischi
tischi.__version__

'0.1.0'

We initially give tischi access to all variables in the current scope.

In [2]:
tischi.initialize(globals())

Turn on debug mode to see what happens

In [3]:
from bia_tischi._machinery import _context
_context.verbose = True

It works using the `%tischi` and `%%tischi` magic commands. Just enter tasks and ask questions. Hint: if you want to ask a question, you need to put a space before the `?`.

In [4]:
a = 5

In [5]:
%tischi generate and execute python code for printing out the variable a

Tools: 6
Variables: 32
Asking for code like this:
 
    Write high-quality python code.
    Use image-processing libraries such as scikit-image, scipy, numpy or pyclesperanto_prototype.
    Do not show results, but save them in variables instead.
    Do not provide additional explanations, just Python code.
    The following variables are available:
    ['In', 'Out', 'a']
    The following functions are available:
    ['get_ipython', 'open', 'tischi']
    
    The code should do the following:
    print(a)

Code:
 print(a)
Execution:
5
Execution done.


The code for printing out the variable "a" was successfully executed.

In [7]:
%tischi generate and execute python code for loading "blobs.tif" and for visualizing it

Tools: 6
Variables: 35
Asking for code like this:
 
    Write high-quality python code.
    Use image-processing libraries such as scikit-image, scipy, numpy or pyclesperanto_prototype.
    Do not show results, but save them in variables instead.
    Do not provide additional explanations, just Python code.
    The following variables are available:
    ['In', 'Out', 'a', 'io']
    The following functions are available:
    ['get_ipython', 'open', 'tischi']
    
    The code should do the following:
    load_image

Code:
 
from skimage import io

# Load the image
image = io.imread('image.jpg')

Execution:


FileNotFoundError: No such file: 'C:\structure\code\bia-tischi\demo\image.jpg'